In [1]:
%load_ext autoreload
%autoreload 2
#%unload_ext autoreload

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
!python -c "import numpy; print(numpy.__version__)"
!python -c "import pandas; print(pandas.__version__)"

In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
#scibert with rules .7366998112933661
#scibert no rules 0.6866199738713892
#bert with rules  0.7386957468427929
#bert no rules 0.6786543765423143
openai = False

    

#file_path = '/Users/gabesmithline/Desktop/gnn_project/data/adj_matrix_scibert_no_rules.pkl'
#file_path = 'adj_matrix.pkl'
file_path = "../gnn_project/papers_missing_all_refs.pkl"
with open(file_path, 'rb') as file:
    G1 = pickle.load(file)
print(G1)
if openai:
    file_path = '../gnn_project/data/adj_matrix_openai_no_rules.pkl'
else:
    file_path = '../gnn_project/data/adj_matrix.pkl'
with open(file_path, 'rb') as file:
    G = pickle.load(file)
print(G.nodes())
    #remove papers in G1 from G
for node in G1:
    if node in G.nodes():
        G.remove_node(node)
        print(f"removed node {node}")
#print node attributes
#for node in G.nodes(data=True):
#     print(*node)




### Convert data

In [5]:
import networkx as nx


In [6]:
#for node in G.nodes(data=True):
#     print(*node)

In [7]:
title_embedding_df = pd.read_csv('data/title_embeddings.csv',index_col=0) if not openai else pd.read_csv('data/title_embeddings_openai.csv',index_col=0)
# title_embedding_df

In [8]:
import os 
if openai: #construct openai embeddings
    paper_embeddings_df = os.path.join("data/paper_embed_openai")
    files = os.listdir(paper_embeddings_df)
    for file in files:
        with open(os.path.join(paper_embeddings_df, file), "rb") as f:
            
            paper_embeddings = np.load(f, allow_pickle=True)
            print(paper_embeddings)




In [ ]:

abs_embedding_df = pd.read_csv('data/abstract_embeddings.csv',index_col=0) if not openai else pd.read_csv('data/abstract_embeddings_openai.csv',index_col=0)
def load_openai_paper_sources():
    import os
    import numpy as np
    import pandas as pd

    data = []
    directory = os.path.join("data/paper_embed_openai")
    files = os.listdir(directory)
    
    for file in files:
        try:
            with open(os.path.join(directory, file), "rb") as f:
                paper_source = np.load(f, allow_pickle=True)
                
                if isinstance(paper_source, (tuple, list)) and len(paper_source) == 2:
                    _id, embedding = paper_source
                elif isinstance(paper_source, dict):
                    _id = paper_source.get('_id', file)
                    embedding = paper_source.get('embedding') or paper_source.get('paper_embeddings', paper_source)
                else:
                    _id = file  
                    embedding = paper_source
                

                if isinstance(_id, str) and _id.endswith(".npy"):
                    _id = _id[:-4] 
                
                data.append({
                    "_id": _id,
                    "paper_embeddings": embedding
                })
        except Exception as e:
            print(f"Failed to load {file}: {e}")
    
    embeddings = pd.DataFrame(data)
    return embeddings


openai_embeddings = load_openai_paper_sources()
print(openai_embeddings.head())
print(openai_embeddings.info())

In [ ]:
for node in G.nodes():
    if node == "6218ac7a5aee126c0f614f1a":
        print(G.nodes[node])


In [ ]:
count = 0
for node in G.nodes():
    print(node)
    if not openai and node in title_embedding_df.index and node in abs_embedding_df.index:
        G.nodes[node]['title_embeddings'] = torch.tensor(title_embedding_df.loc[node].values)
        G.nodes[node]['abstract_embeddings'] = torch.tensor(abs_embedding_df.loc[node].values)
        G.nodes[node]['paper_id'] = node
    elif openai and node in openai_embeddings["_id"].values:
        G.nodes[node]['paper_embeddings'] = torch.tensor(openai_embeddings.loc[openai_embeddings["_id"] == node]["paper_embeddings"].tolist())
        G.nodes[node]['paper_id'] = node
    else:
        G.nodes[node]['paper_id'] = node
        count+=1
print(count)


In [ ]:
node_name_idx_dict = {}
idx = 0
for node in G.nodes(data=True):
    print(node[0])
    node_name_idx_dict[node[0]] = idx
    idx+=1
node_name_idx_dict


In [ ]:
G = nx.relabel_nodes(G, node_name_idx_dict)
G.nodes()[0]

In [14]:
import os
import xml.etree.ElementTree as ET
import pickle
import pandas as pd

import torch.nn.functional as F


In [15]:
def extract_citation_info(xml_path):
   
    import xml.etree.ElementTree as ET

    citation_info = {}
    tree = ET.parse(xml_path)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    paragraphs = root.findall('.//tei:body//tei:p', ns)

    full_text = ''.join(ET.tostring(e, encoding='unicode', method='text') for e in paragraphs)
    total_length = len(full_text)

    current_length = 0  

    for para in paragraphs:
    
        para_text = ET.tostring(para, encoding='unicode', method='text')
        para_length = len(para_text)

        refs_in_para = para.findall('.//tei:ref', ns)

        for ref in refs_in_para:
            ref_id = ref.get('target', '').lstrip('#')
            if not ref_id:
                continue  

        
            position_normalized = current_length / total_length if total_length > 0 else 0.0

            if ref_id not in citation_info:
                citation_info[ref_id] = {
                    'count': 1,
                    'positions': [position_normalized]
                }
            else:
                citation_info[ref_id]['count'] += 1
                citation_info[ref_id]['positions'].append(position_normalized)

        current_length += para_length

    return citation_info

def map_refs_to_paper_ids(xml_path):
   
    ref_to_paper_id = {}
    tree = ET.parse(xml_path)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    bibl_structs = root.findall('.//tei:back//tei:listBibl//tei:biblStruct', ns)
    for bibl in bibl_structs:
        ref_id = bibl.get('{http://www.w3.org/XML/1998/namespace}id')
        if ref_id:
            doi_elem = bibl.find('.//tei:idno[@type="DOI"]', ns)
            if doi_elem is not None and doi_elem.text:
                paper_id = doi_elem.text.strip()
            else:
                title_elem = bibl.find('.//tei:title[@level="a"]', ns)
                if title_elem is not None and title_elem.text:
                    paper_id = title_elem.text.strip()
                else:
                    paper_id = ref_id 
            print(f"ref_id: {ref_id}, paper_id: {paper_id}")
            ref_to_paper_id[ref_id] = paper_id

    return ref_to_paper_id

def get_citation_features(xml_path):
    
    citation_info = extract_citation_info(xml_path)
    ref_to_paper_id = map_refs_to_paper_ids(xml_path)

    citation_features = {}
    for ref_id, info in citation_info.items():
        paper_id = ref_to_paper_id.get(ref_id)
        if paper_id:
            avg_position = sum(info['positions']) / len(info['positions'])
            citation_features[paper_id] = {
                'count': info['count'],
                'avg_position': avg_position
            }
    return citation_features

In [ ]:
all_citation_features = {}


for node in G.nodes(data=True):
    paper_id = node[1]['paper_id']
    xml_path = f'data/paper-xml/{paper_id}.xml'  
    print(xml_path)
    if os.path.exists(xml_path):
        citation_features = get_citation_features(xml_path)
        all_citation_features[paper_id] = citation_features
        print(citation_features)
    else:
        print(f"XML file for {paper_id} does not exist")

    

In [ ]:

paper_ids = [node[1]['paper_id'] for node in G.nodes(data=True)]
print(paper_ids)
paper_id_to_index = {paper_id: idx for idx, paper_id in enumerate(paper_ids)}

edge_counts = []
edge_positions = []
edge_indices = []
for edge in G.edges(data=True):
    print(edge)
    edge_attributes = edge[2]
    src_paper_id, dst_paper_id = edge_attributes["source"], edge_attributes["target"]
    src_idx = paper_id_to_index[src_paper_id]
    dst_idx = paper_id_to_index[dst_paper_id]

    citation_features = all_citation_features.get(src_paper_id, {})
    features = citation_features.get(dst_paper_id, {'count': 0, 'avg_position': 0.0})

    edge_counts.append(features['count'])
    edge_positions.append(features['avg_position'])
    edge_indices.append([src_idx, dst_idx])

edge_index = torch.tensor(edge_indices).t().contiguous()
edge_counts = torch.tensor(edge_counts, dtype=torch.float).view(-1, 1)
edge_positions = torch.tensor(edge_positions, dtype=torch.float).view(-1, 1)

# Concatenate edge attributes
edge_attr = torch.cat([edge_counts, edge_positions], dim=1) 

In [ ]:

title_embedding_df = pd.read_csv('data/title_embeddings.csv', index_col=0)
abstract_embedding_df = pd.read_csv('data/abstract_embeddings.csv', index_col=0)

node_features = []

for node in G.nodes(data=True):
    abstract_embedding = torch.zeros(768, dtype=torch.float)
    title_embedding = torch.zeros(768, dtype=torch.float)
    paper_id = node[1]['paper_id']
    # Get title embedding
    if paper_id in title_embedding_df.index:
        print(f"paper_id {paper_id} found in title_embedding_df")
        title_embedding = torch.tensor(title_embedding_df.loc[paper_id].values, dtype=torch.float)
    else:
        print(f"paper_id {paper_id} not found in title_embedding_df")
        #title_embedding = torch.zeros(768, dtype=torch.float)
    
    if paper_id in abstract_embedding_df.index:
        print(f"paper_id {paper_id} found in abstract_embedding_df")
        abstract_embedding = torch.tensor(abstract_embedding_df.loc[paper_id].values, dtype=torch.float)
    else:
        print(f"paper_id {paper_id} not found in abstract_embedding_df")
        abstract_embedding = torch.zeros(768, dtype=torch.float)
    
    # Concatenate embeddings
    if abstract_embedding.sum() == 0 and title_embedding.sum() == 0:
        print(f"paper_id {paper_id} has no embeddings")
    else:
        combined_features = torch.cat([title_embedding, abstract_embedding])
        node_features.append(combined_features)

node_features_tensor = torch.stack(node_features)
print(node_features_tensor)

In [19]:
from torch_geometric.data import Data

data = Data(
    x=node_features_tensor,
    edge_index=edge_index,
    edge_attr=edge_attr
)

In [20]:
link_label_dict = {}  

edge_labels = []
for idx in range(edge_index.size(1)):
    src_idx = edge_index[0, idx].item()
    dst_idx = edge_index[1, idx].item()
    src_paper_id = G.nodes[src_idx]['paper_id']
    dst_paper_id = G.nodes[dst_idx]['paper_id']

    label = link_label_dict.get((src_paper_id, dst_paper_id), 0)
    edge_labels.append(label)

edge_labels = torch.tensor(edge_labels, dtype=torch.long)
data.edge_label = edge_labels
data.edge_label_index = data.edge_index

In [ ]:

num_edges = data.edge_index.size(1)
num_val = int(num_edges * 0.1)
num_test = int(num_edges * 0.2)
num_train = num_edges - num_val - num_test


perm = torch.randperm(num_edges)
train_idx = perm[:num_train]
val_idx = perm[num_train:num_train + num_val]
test_idx = perm[num_train + num_val:]

#CHECK IF DATA CONTAINS NANs
if torch.isnan(data.x).any():
    print("NaN detected in node features")
else:
    print("No NaN in node features")
if torch.isnan(data.edge_index).any():
    print("NaN detected in edge index")
else:
    print("No NaN in edge index")
if torch.isnan(data.edge_attr).any():
    print("NaN detected in edge attributes")
else:
    print("No NaN in edge attributes")
from copy import deepcopy

train_data = deepcopy(data)
train_data.edge_index = data.edge_index[:, train_idx]
train_data.edge_attr = data.edge_attr[train_idx]
train_data.edge_label = data.edge_label[train_idx]
train_data.edge_label_index = data.edge_index[:, train_idx]

val_data = deepcopy(data)
val_data.edge_index = data.edge_index[:, val_idx]
val_data.edge_attr = data.edge_attr[val_idx]
val_data.edge_label = data.edge_label[val_idx]
val_data.edge_label_index = data.edge_index[:, val_idx]

test_data = deepcopy(data)
test_data.edge_index = data.edge_index[:, test_idx]
test_data.edge_attr = data.edge_attr[test_idx]
test_data.edge_label = data.edge_label[test_idx]
test_data.edge_label_index = data.edge_index[:, test_idx]


In [ ]:
def normalize_edge_attributes(edge_attr):
    """
    Normalizes edge attributes to have zero mean and unit variance.
    """
    mean = edge_attr.mean(dim=0, keepdim=True)
    std = edge_attr.std(dim=0, keepdim=True)
    std[std < 1e-6] = 1.0  # Prevent division by zero
    return (edge_attr - mean) / std

#edge_attr = normalize_edge_attributes(edge_attr)

if torch.isnan(edge_attr).any() or torch.isinf(edge_attr).any():
    raise ValueError("Invalid values detected in edge_attr after normalization.")

print("Edge attributes normalized successfully.")

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [24]:

from torch.nn import Module
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCNModel(Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_attr_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)
        
        edge_mlp_input_dim = out_channels * 2 + edge_attr_dim  # 2*128 + 2 = 258
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(edge_mlp_input_dim, 64),
            torch.nn.LeakyReLU(0.1),
            torch.nn.Linear(64, 1) 
        )
        
    def forward(self, data):
        z = self.encode(
            data.x.to(device), 
            data.edge_index.to(device), 
            data.edge_attr.to(device)
        )
        
        out = self.decode(
            z, 
            data.edge_label_index.to(device), 
            data.edge_attr.to(device)
        )

        out = self.dropout(out)
        
        return out
    
    def normalize_node_features(self, x):
        print("Before normalization - min:", x.min().item(), "max:", x.max().item(), "mean:", x.mean().item())
        
        mean = x.mean(dim=0, keepdim=True)
        std = x.std(dim=0, keepdim=True)
        std[std < 1e-6] = 1.0 
        x_normalized = (x - mean) / std
        x_normalized = torch.clamp(x_normalized, min=-5.0, max=5.0)
        
        print("After normalization - min:", x_normalized.min().item(), "max:", x_normalized.max().item(), "mean:", x_normalized.mean().item())
        
        if torch.isnan(x_normalized).any():
            print("NaNs detected in normalized x")
        return x_normalized
    
    def encode(self, x, edge_index, edge_attr):
        x = self.normalize_node_features(x)
        
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.1)  # Using LeakyReLU
        x = torch.clamp(x, min=-10.0, max=10.0)
        
        if torch.isnan(x).any():
            print("NaN detected after conv1 and activation/clamp")
            return x  # Early return for debugging
        
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.1)  # Using LeakyReLU
        # Clamping to prevent exploding values
        x = torch.clamp(x, min=-10.0, max=10.0)
        
        if torch.isnan(x).any():
            print("NaN detected after conv2 and activation/clamp")
            return x  
        
        return x

    def decode(self, z, edge_label_index, edge_attr):
        src = z[edge_label_index[0]]  
        dst = z[edge_label_index[1]]  
        
        print(f"src shape: {src.shape}")
        print(f"dst shape: {dst.shape}")
        print(f"edge_attr shape: {edge_attr.shape}")
        print(f"edge_label_index shape: {edge_label_index.shape}")
        
        if edge_attr.size(0) != edge_label_index.size(1):
            repeat_times = edge_label_index.size(1) // edge_attr.size(0)
            remainder = edge_label_index.size(1) % edge_attr.size(0)
            
            if remainder == 0:
                edge_attr = edge_attr.repeat(repeat_times, 1)
            else:
                edge_attr = torch.cat([
                    edge_attr.repeat(repeat_times, 1),
                    edge_attr[:remainder]
                ], dim=0)
        
        # Verify shapes after adjustment
        assert src.size(0) == dst.size(0) == edge_attr.size(0), \
            f"Shape mismatch: src={src.shape}, dst={dst.shape}, edge_attr={edge_attr.shape}"
        
        combined = torch.cat([src, dst, edge_attr], dim=1)
        score = self.edge_mlp(combined)
        return score.view(-1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCNModel(
    in_channels=1536,        
    hidden_channels=128,
    out_channels=128,
    edge_attr_dim=2          
).to(device)

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu')
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)
        elif isinstance(m, GCNConv):
            torch.nn.init.kaiming_uniform_(m.lin.weight, nonlinearity='leaky_relu')
            if m.lin.bias is not None:
                torch.nn.init.zeros_(m.lin.bias)
        elif isinstance(m, torch.nn.BatchNorm1d):
            torch.nn.init.ones_(m.weight)
            torch.nn.init.zeros_(m.bias)

# After model creation
initialize_weights(model)
print("Weights initialized successfully.")

In [ ]:
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data

data = Data(
    x=node_features_tensor,
    edge_index=edge_index,
    edge_attr=edge_attr,
)

print("Before split:")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Max edge index: {data.edge_index.max().item()}")


def verify_and_fix_data(data):
    num_nodes = data.x.size(0)
    valid_edges_mask = (data.edge_index[0] < num_nodes) & (data.edge_index[1] < num_nodes)
    
    data.edge_index = data.edge_index[:, valid_edges_mask]
    data.edge_attr = data.edge_attr[valid_edges_mask]
    
    return data

data = verify_and_fix_data(data)

print("\nAfter verifying and fixing edge indices:")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Max edge index: {data.edge_index.max().item()}")

transform = RandomLinkSplit(
    num_val=0.1,          # 10% for validation
    num_test=0.2,         # 20% for testing
    is_undirected=True,  
    add_negative_train_samples=True
)

train_data, val_data, test_data = transform(data)

print(f"\nOriginal number of nodes in G: {G.number_of_nodes()}")
print(f"Original number of edges in G: {G.number_of_edges()}")

print("\nAfter split:")
print(f"Train data - Nodes: {train_data.num_nodes}, Edges: {train_data.num_edges}, Max edge index: {train_data.edge_index.max().item()}")
print(f"Val data - Nodes: {val_data.num_nodes}, Edges: {val_data.num_edges}, Max edge index: {val_data.edge_index.max().item()}")
print(f"Test data - Nodes: {test_data.num_nodes}, Edges: {test_data.num_edges}, Max edge index: {test_data.edge_index.max().item()}")

assert train_data.edge_index.max().item() < train_data.num_nodes, "Invalid edge indices in train data"
assert val_data.edge_index.max().item() < val_data.num_nodes, "Invalid edge indices in val data"
assert test_data.edge_index.max().item() < test_data.num_nodes, "Invalid edge indices in test data"

print("\nData splitting successful with consistent edge indices.")




In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
pos_weight = (train_data.edge_label == 0).sum() / (train_data.edge_label == 1).sum()
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))

In [28]:
def train(data):
    model.train()
    optimizer.zero_grad()
    
    eps = 1e-7
    
    out = model(data)
    out = torch.clamp(out, min=1e-7, max=1-1e-7)  # Prevent extreme values
        
    print(f"Output range: min={out.min().item()}, max={out.max().item()}")
    print(f"Labels range: min={data.edge_label.min().item()}, max={data.edge_label.max().item()}")
 
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
    individual_losses = criterion(out, data.edge_label.float().to(device))

    if torch.isnan(individual_losses).any():
        print("NaN detected in individual losses")
        individual_losses = torch.nan_to_num(individual_losses, nan=0.0)

    loss = individual_losses.mean()
    loss.backward()
  
    for name, param in model.named_parameters():
        if param.grad is not None:
            param.grad = torch.nan_to_num(param.grad, nan=0.0, posinf=1.0, neginf=-1.0)
    
    optimizer.step()
    return loss.item()

In [29]:

@torch.no_grad()
def evaluate(data):
    model.eval()
    data = data.to(device)  
    
    z = model.encode(data.x, data.edge_index, data.edge_attr)
    

    if torch.isnan(z).any():
        print("NaN detected in node embeddings z")
        return 0.0, 0.0
    else:
        print("No NaN in node embeddings z")
    
    out_raw = model.decode(z, data.edge_label_index, data.edge_attr)
    
    if torch.isnan(out_raw).any():
        print("NaN detected in raw output of model.decode")
        return 0.0, 0.0
    else:
        print("No NaN in raw output of model.decode")
    
    out = out_raw.sigmoid()
    
    if torch.isnan(out).any():
        print("NaN detected in output after sigmoid")
        return 0.0, 0.0
    else:
        print("No NaN in output after sigmoid")
    
    print(f"Shape of out: {out.shape}") 
    print(f"Shape of data.edge_label: {data.edge_label.shape}")  
    
    if out.shape != data.edge_label.shape:
        print("Shape mismatch between predictions and labels.")
        return 0.0, 0.0
    
    pred = out >= 0.5
    acc = pred.eq(data.edge_label.to(device).bool()).sum().item() / data.edge_label.size(0)
    
    from sklearn.metrics import average_precision_score
    y_true = data.edge_label.cpu().numpy()
    y_scores = out.cpu().detach().numpy()
    
    if np.isnan(y_true).any():
        print("NaN detected in y_true")
        y_true = np.nan_to_num(y_true, nan=0.0)
    
    if np.isnan(y_scores).any():
        print("NaN detected in y_scores")
        y_scores = np.nan_to_num(y_scores, nan=0.0)
    
    map_score = average_precision_score(y_true, y_scores)
    
    return acc, map_score

In [ ]:
print(f"Initial number of nodes: {node_features_tensor.shape[0]}")
print(f"Initial edge_index shape: {edge_index.shape}")
print(f"Max node index in edge_index: {edge_index.max().item()}")

# Get unique node indices from edge_index
unique_nodes_in_edges = torch.unique(edge_index)
print(f"Unique nodes in edge_index: {len(unique_nodes_in_edges)}")

num_nodes = node_features_tensor.shape[0]
valid_nodes = set(range(num_nodes))
nodes_in_edges = set(unique_nodes_in_edges.tolist())
valid_edge_nodes = nodes_in_edges.intersection(valid_nodes)

node_idx_map = {old_idx: new_idx for new_idx, old_idx in enumerate(sorted(valid_edge_nodes))}

valid_edges_mask = torch.tensor([
    (edge_index[0, i].item() in node_idx_map) and (edge_index[1, i].item() in node_idx_map)
    for i in range(edge_index.shape[1])
])

edge_index = edge_index[:, valid_edges_mask]
edge_attr = edge_attr[valid_edges_mask]

edge_index_list = edge_index.tolist()
edge_index = torch.tensor([
    [node_idx_map[src] for src in edge_index_list[0]],
    [node_idx_map[dst] for dst in edge_index_list[1]]
])

new_node_features = torch.stack([node_features_tensor[i] for i in sorted(valid_edge_nodes)])

# Update node_features_tensor
node_features_tensor = new_node_features

# Verify the fix
print(f"\nAfter fixing:")
print(f"Number of nodes: {node_features_tensor.shape[0]}")
print(f"Edge index shape: {edge_index.shape}")
print(f"Max node index in edge_index: {edge_index.max().item()}")
assert edge_index.max() < node_features_tensor.shape[0], "Edge indices still contain invalid references"

In [ ]:
epochs = 1000
best_loss = float('inf')
loss_list = []
acc_list = []
map_list = []
test_acc_list = []
test_map_list = []

for epoch in range(1, epochs + 1):
   #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)  # Reduced from 1.0
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)  # Added weight decay
    
    loss = train(train_data)
    
    if torch.isnan(torch.tensor(loss)) or torch.isinf(torch.tensor(loss)):
        print(f"Epoch {epoch}: Loss is {loss}. Stopping training.")
        break
    
    if loss < best_loss:
        print(f"New best loss: {loss:.4f}")
        best_loss = loss
    
    print(f"Epoch {epoch} - Loss: {loss:.4f}")
    loss_list.append(loss)
    
    val_acc, val_map = evaluate(val_data)
    acc_list.append(val_acc)
    map_list.append(val_map)
    test_acc, test_map = evaluate(test_data)
    test_acc_list.append(test_acc)
    test_map_list.append(test_map)
    print(f'Metrics - Val Acc: {val_acc:.4f}, Val MAP: {val_map:.4f}, Test Acc: {test_acc:.4f}, Test MAP: {test_map:.4f}')

In [ ]:
import matplotlib.pyplot as plt
from IPython import display

plt.figure(figsize=(10, 6))
plt.plot(loss_list, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True)

# Display the plot
display.display(plt.gcf())
plt.close()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(acc_list, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Over Time')
plt.legend()
plt.grid(True)
display.display(plt.gcf())
plt.close()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(map_list, label='Validation MAP')
plt.xlabel('Epoch')
plt.ylabel('MAP')
plt.title('Validation MAP Over Time')
plt.legend()
plt.grid(True)
display.display(plt.gcf())
plt.close()

In [35]:
def custom_map(predictions, ground_truths):

    combined = list(zip(predictions, ground_truths))
    print("Combined Predictions and Ground Truths:", combined)
    
    sorted_combined = sorted(combined, key=lambda x: x[0], reverse=True)
    print("Sorted Combined List:", sorted_combined)
    
    den = len(sorted_combined)
    print("Total Number of Predictions (den):", den)
    
    score = 0
    
    for rank, (_, gt) in enumerate(sorted_combined, start=1):
        if gt == 1:  
            contribution = 1 - (rank - 1) / den
            score += contribution
            print(f"Rank: {rank}, GT: {gt}, Contribution: {contribution}, Cumulative Score: {score}")
    
    num_positives = sum(ground_truths) 
    print("Number of Positives:", num_positives) 
    #if there are no positive edges, return 0, the number of ground truth should be at most the number of predictions
    map_score = score / min(len(predictions), num_positives) if num_positives > 0 else 0
    print("Calculated MAP Score:", map_score)
    
    return map_score


In [ ]:
test_predictions = model(test_data)
print(f"test_predictions: {len(test_predictions)}")

In [ ]:
test_predictions_ground_truth = test_data.edge_label.cpu().numpy()[:len(test_predictions)]
map_score = custom_map(test_predictions, test_predictions_ground_truth)
print(f"Custom MAP score: {map_score:.4f}") 
torch.save(test_predictions, 'test_predictions.pt')
